In [1]:
!pip install tensorflow-text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.2
    Uninstalling tensorflow-2.16.2:
      Successfully uninstalled tensorflow-2.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.16.0 requires tensorflow<2.17,>=2.16, but you have tensorflow 2.18.0 w

In [2]:
!pip install tensorflow tensorflow_hub tensorflow_text --upgrade


INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.16.0
    Uninstalling tf_keras-2.16.0:
      Successfully uninstalled tf_keras-2.16.0


In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as tf_text

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity

ImportError: /usr/local/lib/python3.11/dist-packages/tensorflow_text/core/pybinds/pywrap_fast_bert_normalizer_model_builder.so: undefined symbol: _ZN3tsl8internal10LogMessageC1EPKciN4absl12lts_2023080211LogSeverityE

In [ ]:
df = pd.read_csv('Spam_SMS.csv')

In [ ]:
df.head(5)

In [ ]:
df.groupby('Class').describe()

In [ ]:
747/4827
# so there are 15% spam emails onlt so we will now do downsampling , to disregard some ham emails so that we can equalize the ratio of spam and ham emails......


In [ ]:
# split spam and ham email
df_spam = df[df["Class"]=='spam']
df_ham = df[df["Class"]=='ham']
df_spam.shape , df_ham.shape

In [ ]:
df_spam.shape[0] # means 1st outside dimension

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

In [ ]:
#now concatt df_han_downsampled with df_spam

df_balanced = pd.concat([df_spam,df_ham_downsampled])
df_balanced.shape

In [ ]:
#now create a new spam column

df_balanced['spam'] = df_balanced['Class'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embeding(sentences):

    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']
    # it bert_encoder returns an dictionary in wich pooled_output is a key , we can see multiple keys of this dict too

# get_sentence_embeding(["hellow worlf how are thou soul"])

In [ ]:
e = get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

e # e is embedding of all these words

In [ ]:
cosine_similarity([e[0]] , [e[1]]) # it expects 2 dimensional list thats why we put [e[0]] in it

cosine_similarity([e[0]] , [e[3]])
# the cosine similiarty shows the similarity between both by using the wikipedia text.......

cosine_similarity([e[5]] , [e[3]])

In [ ]:
#bert layers

text_input = tf.keras.layers.Input(shape=(10,), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
#neural network layers

l = tf.keras.layers.Dropout(0.1, name="dropout")(output['pooled_output'])
# the Dropout will take the output of bert_encoder with pooling_output key as input of this layer

tf.keras.Dense(1,activation="sigmoid")(l) # 1 bcz it would be spam or not spam

#construvt fianl model

model = tf.keras.Model(inputs=[text_input] ,outputs=[l] )

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.evaluate(X_test , y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()